In [ ]:
%pip install chatterbot
%pip install chatterbot_corpus

In [ ]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
print("Required modules are successfully imported...")

Create object of ChatBot class

In [ ]:
my_bot = ChatBot(name='MyBot', read_only=True,
                 logic_adapters=['chatterbot.logic.MathematicalEvaluation',
                                 'chatterbot.logic.BestMatch'])

Loading data

In [ ]:
talk1 = ['hi there!',
              'hi!',
              'how do you do?',
              'Cool as always'
              'how are you?',
              'i\'m cool.',
              'fine, you?',
              'always cool.',
              'i\'m ok',
              'glad to hear that.',
              'i\'m fine',
              'glad to hear that.',
              'i feel awesome',
              'excellent, glad to hear that.',
              'good',
              'Thats awesome to hear'
              'not so good',
              'sorry to hear that.',
              'I need your assistance regarding my order',
              'Please, Provide me with your order id',
              'I have a complaint.',
              'Please elaborate, your concern',
              'How long it will take to receive an order ?',
              'An order takes 3-5 Business days to get delivered.',
              'The product delivered is damaged',
              'Oh! we are really sorry for! please provide your order id we will replace it for you',
              'Okay Thanks',
              'No Problem! Have a Good Day!'
              'what\'s your name?',
              'i\'m your bot. Ask me a math question, please.']

In [ ]:
talk2 = ['pythagorean theorem',
               'a squared plus b squared equals c squared.']

Training our model

In [ ]:
list_trainer = ListTrainer(my_bot)
for item in (talk1, talk2):
    list_trainer.train(item)
print("Training of chatbot is completed")

Testing with input

In [ ]:
print(my_bot.get_response("how are you"))

In [ ]:
name=input("Enter Your Name: ")
print("Hey welcome " + name + "! Let's chat!!!")
while True:
    request=input(name+':')
    if request=='Bye' or request =='bye':
        print('Bot: Ok Bye. See you then!!')
        break
    else:
        response=my_bot.get_response(request)
        print('Bot:',response)

Chatterbot Corpus

In [ ]:
from chatterbot.trainers import ChatterBotCorpusTrainer
corpus_trainer = ChatterBotCorpusTrainer(my_bot)
corpus_trainer.train('chatterbot.corpus.english')

In [ ]:
print(my_bot.get_response("tell me a joke"))

In [ ]:
name=input("Enter Your Name: ")
print("Welcome to the Bot Service! Let me know how can I help you?")
while True:
    request=input(name+':')
    if request=='Bye' or request =='bye':
        print('Bot: Bye')
        break
    else:
        response=my_bot.get_response(request)
        print('Bot:',response)

# NLTK

We are going to build a simple chatbot using nltk library

( Natural Language Toolkit). 

It is a leading platform for building Python programs to work with human language data.

Importing Required Libraries

In [ ]:
import numpy as np
import random
import json
import nltk
from nltk.stem.porter import PorterStemmer #stemming convert words to root words

In [ ]:
nltk.download('punkt')  #tokenize split sentence into words

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, words):
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1
    return bag

In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        return out

In [ ]:
with open('intents.json', 'r') as f:
    intents = json.load(f)

In [ ]:
all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

In [ ]:
# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags),"tags")
print(len(all_words),"words")

In [ ]:
print(xy)

In [ ]:
print(tags)

In [ ]:
print(all_words)

Training

In [ ]:
# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
# Hyper-parameters 
num_epochs = 1000
batch_size = 8  
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

In [ ]:
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

In [ ]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print(f'final loss: {loss.item():.4f}')

In [ ]:
data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'Training complete. File saved to {FILE}')

In [ ]:
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model.load_state_dict(model_state)
model.eval()

In [ ]:
bot_name = "Doremon"
print("Let's chat! (type 'quit' to exit)")
while True:
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")